##**TRANSLATION WITH A SEQUENCE TO SEQUENCE NETWORK WITH ATTENTION**   
> Sequence to Sequence network for translating English to Hindi and Hindi to English

This code contains two parts  
  - Preprocessing dataset, training two networks and save models (In `train_and_save()` funtion)
  - Load saved models with trimmed dataset and evaluate random examples (In `sample_evaluate()` function)
*******************************************************
**Files**
 - Training/Testing dataset : [trimmed_data.csv](https://raw.githubusercontent.com/ashans/eng-hindi-translator/main/output/trimmed_data.csv)
 - Pickled files
   - English to Hindi Encoder : [eng_to_hin_encoder.pkl](https://raw.githubusercontent.com/ashans/eng-hindi-translator/main/output/eng_to_hin_encoder.pkl)
   - English to Hindi Decoder : [eng_to_hin_decoder.pkl](https://raw.githubusercontent.com/ashans/eng-hindi-translator/main/output/eng_to_hin_decoder.pkl)
   - Hindi to English Encoder : [hin_to_eng_encoder.pkl](https://raw.githubusercontent.com/ashans/eng-hindi-translator/main/output/hin_to_eng_encoder.pkl)
   - Hindi to English Decoder : [hin_to_eng_decoder.pkl](https://raw.githubusercontent.com/ashans/eng-hindi-translator/main/output/hin_to_eng_decoder.pkl)

**Pipeline method**
- `sample_evaluate()`
*******************************************************
*Original Dataset - [https://www.manythings.org/anki/hin-eng.zip](https://www.manythings.org/anki/hin-eng.zip)*  
*Code Reference - [NLP FROM SCRATCH: TRANSLATION WITH A SEQUENCE TO SEQUENCE NETWORK AND ATTENTION](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)*  
*Github - [https://github.com/ashans/eng-hindi-translator](https://github.com/ashans/eng-hindi-translator)*

**Requirements**

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import urllib

import torch
import torch.nn as nn
from torch.utils import model_zoo
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using - %s" %device)

Using - cpu


Creating helper class for `word2index` and `index2word`

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Splitter and character replacing

In [3]:
def splitNormalize(s):
      splitted = s.split('\t')[:2]
      splitted[0] = re.sub(r"([.!?])", r" \1", splitted[0].lower().strip())
      splitted[0] = re.sub(r"[^a-zA-Z.!?]+", r" ", splitted[0])

      splitted[1] = re.sub(r"([.!?।])", r" \1", splitted[1].strip())

      return splitted[:2]

Loading dataset

In [4]:
DATA_URL = "https://raw.githubusercontent.com/ashans/eng-hindi-translator/main/dataset/hin.txt"
def load_content():
  file = urllib.request.urlopen(DATA_URL)
  
  s = []
  for line in file:
    s.append(splitNormalize(line.decode("utf-8")))
  print("Loaded sample data - %s" % s[0])
  return s

Filtering samples from dataset
> Only samples with limited number of words and types are selected

In [5]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
def prepareData():
    pairs = load_content()
    eng_lang = Lang('eng')
    hin_lang = Lang('hin')
    print("Read %s sentence pairs" % len(pairs))
    print("Sample data - %s" % pairs[0])
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        eng_lang.addSentence(pair[0])
        hin_lang.addSentence(pair[1])
    print("Counted words:")
    print(eng_lang.name, eng_lang.n_words)
    print(hin_lang.name, hin_lang.n_words)
    return eng_lang, hin_lang, pairs

Encoder

In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Decoder with Attention

In [8]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair, input_lang, output_lang):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training function with **teacher forcing**

In [10]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

Helper to print elapsed time

In [11]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.

In [12]:
def trainIters(input_lang, output_lang, pairs, encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs), input_lang, output_lang)
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [13]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation function

In [14]:
def evaluate(input_lang, output_lang, encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                # decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [15]:
def save_model(model, name):
    torch.save(model, f'{name}.pkl')

In [16]:
import csv
def save_data(data, name):
    # open the file in the write mode
    with open(f'{name}.csv', 'w') as f:
        # create the csv writer
        writer = csv.writer(f)

        # write a row to the csv file
        for line in data:
            writer.writerow(line)

**Train the models and save outputs**

In [17]:
hidden_size = 256
def train_and_save():  
    eng_lang, hin_lang, eng_hindi_pairs = prepareData()

    print("Saving trimmed data")
    save_data(eng_hindi_pairs, "trimmed_data")

    # Train English to Hindi translation
    print("Training English to Hindi translation")
    encoder1 = EncoderRNN(eng_lang.n_words, hidden_size).to(device)
    attn_decoder1 = AttnDecoderRNN(hidden_size, hin_lang.n_words, dropout_p=0.1).to(device)
    trainIters(eng_lang, hin_lang, eng_hindi_pairs, encoder1, attn_decoder1, 15000, print_every=5000)

    # Save English to Hindi translation models
    print("Saving English to Hindi translation models")
    save_model(encoder1, 'eng_to_hin_encoder')
    save_model(attn_decoder1, 'eng_to_hin_decoder')

    # Train Hindi to English translation
    print("Training Hindi to English translation")
    encoder2 = EncoderRNN(hin_lang.n_words, hidden_size).to(device)
    attn_decoder2 = AttnDecoderRNN(hidden_size, eng_lang.n_words, dropout_p=0.1).to(device)
    trainIters(hin_lang,eng_lang, [list(reversed(p)) for p in eng_hindi_pairs], encoder2, attn_decoder2, 15000, print_every=5000)

    # Save Hindi to English translation models
    print("Saving Hindi to English translation models")
    save_model(encoder2, 'hin_to_eng_encoder')
    save_model(attn_decoder2, 'hin_to_eng_decoder')

    print("Complete")

In [28]:
def load_model(url):
  model = model_zoo.load_url(f'https://raw.githubusercontent.com/ashans/eng-hindi-translator/main/output/{url}')
  # model = torch.load(url)
  return model

In [35]:
def load_trimmed_data():
  file = urllib.request.urlopen('https://raw.githubusercontent.com/ashans/eng-hindi-translator/main/output/trimmed_data.csv')
  
  s = []
  for line in file:
    s.append(line.decode("utf-8").strip().split(','))

  return s

In [26]:
def load_models():
  eng_to_hin_encoder = load_model('eng_to_hin_encoder.pkl')
  eng_to_hin_decoder = load_model('eng_to_hin_decoder.pkl')
  hin_to_eng_encoder = load_model('hin_to_eng_encoder.pkl')
  hin_to_eng_decoder = load_model('hin_to_eng_decoder.pkl')

  return eng_to_hin_encoder, eng_to_hin_decoder, hin_to_eng_encoder, hin_to_eng_decoder

In [21]:
def translate(input_lang, output_lang, encoder, decoder, sentence):
  output_words, _ = evaluate(input_lang, output_lang, encoder, decoder, sentence)
  return ' '.join(output_words)

In [41]:
import pandas as pd
def show_table(data):
  table = pd.DataFrame(data, columns = ['Original English', 'Original Hindi', 'Engish -> Hindi', 'English -> Hindi -> English'])

  display(table)

**Load `pkl` files to models and evaluate given number of samples**

In [43]:
def sample_evaluate(no_of_samples = 10):
  e_to_h_encoder, e_to_h_decoder, h_to_e_encoder, h_to_e_decoder = load_models()
  trimmed_data = load_trimmed_data()
  eng_lang, hin_lang, _ = prepareData()
  result = []
  for i in range(no_of_samples):
    selected = random.choice(trimmed_data)
    hindi_sentence = translate(eng_lang, hin_lang, e_to_h_encoder, e_to_h_decoder, selected[0])
    english_sentence = translate(hin_lang, eng_lang, h_to_e_encoder, h_to_e_decoder, hindi_sentence)
    result.append([selected[0], selected[1], hindi_sentence, english_sentence])
  print('---------------- Results -----------------')
  show_table(result)

In [ ]:
# train_and_save() # Uncomment to train two pairs of encoder/decoder and save trimmed dataset and pkl files

In [44]:
# This will load the pkl files and trimmed dataset from github and evaluate provided number of random examples
sample_evaluate(10)

Loaded sample data - ['wow !', 'वाह !']
Read 2952 sentence pairs
Sample data - ['wow !', 'वाह !']
Trimmed to 179 sentence pairs
Counting words...
Counted words:
eng 293
hin 338
---------------- Results -----------------


,Original English,Original Hindi,Engish -> Hindi,English -> Hindi -> English
0,i m disappointed with you .,मैं तुमसे निराश हूँ ।,मैं तुमसे निराश हूँ ।,i m disappointed with you .
1,i m not very old .,मैं बहुत बूढ़ी नहीं हूँ ।,मैं बहुत बूढ़ी नहीं हूँ ।,i m not very old .
2,i m busy with my homework .,मैं अपने होमवर्क में लगा हुआ हूँ ।,मैं अपने होमवर्क में लगा हुआ हूँ ।,i m busy with my homework .
3,she is poor but happy .,"""वह गरीब है","वह गरीब है, पर खुश है ।",he is poor but happy .
4,i am confident he will keep his promise .,मुझे भरोसा है कि वह अपना वादा निभाएगा ।,मुझे भरोसा है कि वह अपना वादा निभाएगा ।,i am confident he will keep his promise .
5,i m going to tell you something .,मैं तुम्हें कुछ बताने जा रहा हूँ ।,मैं तुम्हें कुछ बताने वाला हूँ ।,i m going to tell you something .
6,i am his friend and will remain so .,मैं उसका दोस्त हूँ और रहूँगा ।,मैं उसका दोस्त हूँ और रहूँगा ।,i am his friend and will remain so .
7,we re against war .,हम जंग के ख़िलाफ़ हैं ।,हम युद्ध का विरोध करते हैं ।,we re against war .
8,she is not afraid of anything .,वह किसी भी चीज़ से नहीं डरती है ।,वह किसी भी चीज़ से नहीं डरती है ।,she is not afraid of anything .
9,she is a wealthy woman .,वह बहुत पैसेवाली औरत है ।,वह बहुत पैसेवाली औरत है ।,she is a wealthy woman .
